# Test Env

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from typing import Optional, List, Annotated, Dict, Sequence

from dataclasses import dataclass
from spytial import *


In [2]:
from dataclasses import dataclass, field
from pathlib import Path

CHILDREN_SELECTOR = "{p, c : Node | c in p.children.idx[int]}"

SAME_PARENT_SELECTOR = "{ c1, c2 : Node | (c1 != c2) and (some p : Node |    c1 in p.children.idx[int] and    c2 in p.children.idx[int]) }"

@dataclass
@attribute(field="is_dir")
@attribute(field="path")
@hideAtom(selector="int + str + list")
# We don't actually care about the ORDER of children, so maybe group them?
@group(selector=CHILDREN_SELECTOR, name="children", addEdge=True)
@orientation(selector=CHILDREN_SELECTOR, directions=["right"])
@align(selector=SAME_PARENT_SELECTOR, direction="vertical")
@icon(selector="{n : Node | n.is_dir = True}", path="fa:folder", showLabels=True)
@atomColor(selector="{n : Node | n.is_dir = True}", value="black")
class Node:
    path: str          # root-relative, as a plain string
    is_dir: bool
    children: list["Node"] = field(default_factory=list)

def snapshot_tree(root: str, max_depth: int | None = None) -> Node:
    rootp = Path(root).resolve()
    base_depth = len(rootp.parts)

    def build(p: Path) -> Node:
        rel = p.relative_to(rootp).as_posix()   # stable "/" separators
        depth = len(p.parts) - base_depth
        n = Node(path=rel, is_dir=p.is_dir())

        if n.is_dir and (max_depth is None or depth < max_depth):
            for child in sorted(p.iterdir(), key=lambda x: x.name):
                n.children.append(build(child))

        return n

    root_node = Node(path=".", is_dir=True)
    for child in sorted(rootp.iterdir(), key=lambda x: x.name):
        root_node.children.append(build(child))

    return root_node



t = snapshot_tree("/Users/siddharthaprasad/Documents/ttt/demo-fs", max_depth=3)
print(t)
diagram(t, height=800)


Node(path='.', is_dir=True, children=[Node(path='README.md', is_dir=False, children=[]), Node(path='data', is_dir=True, children=[Node(path='data/cleaned.csv', is_dir=False, children=[]), Node(path='data/raw.csv', is_dir=False, children=[])]), Node(path='latest.csv', is_dir=False, children=[]), Node(path='pyproject.toml', is_dir=False, children=[]), Node(path='src', is_dir=True, children=[Node(path='src/demoapp', is_dir=True, children=[Node(path='src/demoapp/__init__.py', is_dir=False, children=[]), Node(path='src/demoapp/main.py', is_dir=False, children=[]), Node(path='src/demoapp/utils.py', is_dir=False, children=[])])]), Node(path='tests', is_dir=True, children=[Node(path='tests/test_main.py', is_dir=False, children=[])])])
